# 3.3.2 Image Augmentation: Cutout, MixUp and CutMix
By Zac Todd

This tutorials covers the image augmenations included in the DeVries and Taylors work [Cutout](https://arxiv.org/abs/1708.04552), Zhang et al work [MixUp](https://arxiv.org/abs/1710.09412) and Yun et al work [CutMix](https://arxiv.org/abs/1905.04899). 

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image, ImageOps, ImageEnhance

IMAGES_DIR = f'{os.getcwd()}/resources'
IMAGE_1 = f'{IMAGES_DIR}/dog.jpg'
IMAGE_2 = f'{IMAGES_DIR}/cat.jpg'
IMAGE_3 = f'{IMAGES_DIR}/cat_on_dog.jpg'

Wrapper for enabling functions that take np.ndarray to take PIL.Image as input and then after running converts the output np.ndarray to PIL.Image dont worry to much about what this function is doing.

In [ ]:
def _PIL_NUMPY(func):
    def wrapper(*args, **kwargs):
        new_args = [np.asarray(arg) if isinstance(arg, type(Image.Image())) else arg for arg in args]
        new_kwargs = {k: (np.asarray(arg) if isinstance(arg, type(Image.Image())) else arg) for k, arg in kwargs.items()}
        out_array = func(*new_args, **new_kwargs)
        out_image = Image.fromarray(np.uint8(out_array))
        return out_image
    return wrapper

## Cutout
Cutout removes a selected number of square holes from and image.

In [ ]:
@_PIL_NUMPY
def cutout(image, holes, length):
    output = image.copy()
    h, w, _ = output.shape
    for _ in range(holes):
        x0, y0 = np.random.randint(w - length), np.random.randint(h - length)
        output[y0: y0 + length, x0:x0 + length] = 0
    return output
    
img = Image.open(IMAGE_1)
cutout_image = cutout(img, 10, 500)
cutout_image

Now instead of cuting out with black space try cuting out the image with unifrom noise. 
Hint look at *np.random.randint*.

In [ ]:
@_PIL_NUMPY
def noisy_cutout(image, holes, length):
    output = image.copy()
    h, w, _ = output.shape
    for _ in range(holes):
        x0, y0 = np.random.randint(w - length), np.random.randint(h - length)
        output[y0:y0 + length, x0:x0 + length] = ...
    return output
    
img = Image.open(IMAGE_2)
noisy_cutout_image = noisy_cutout(img, 10, 500)
noisy_cutout_image

<script type="text/x-mathjax-config">
MathJax.Hub.Config({
tex2jax: {
inlineMath: [['$','$'], ['\\(','\\)']],
processEscapes: true},
jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
TeX: {
extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
equationNumbers: {
autoNumber: "AMS"
}
}
});
</script>
The Cutout function from the DeVries and Taylors samples boxes a bit differently than in our *cutout(.)* function. Rewrite the function so the we can sample $x_0$ and $y_0$ from just the bounds of the image.

In [ ]:
@_PIL_NUMPY
def cutout2(image, holes, length):
    output = image.copy()
    h, w, _ = output.shape
    for _ in range(holes):
        x0, y0 = np.random.randint(w), np.random.randint(h)
        x1, y1 = ...
        output[y0: x1, x0: y1] = 0
    return output

img = Image.open(IMAGE_2)
cutout2_image = cutout2(img, 10, 500)
cutout2_image

## Mixup

<script type="text/x-mathjax-config">
MathJax.Hub.Config({
tex2jax: {
inlineMath: [['$','$'], ['\\(','\\)']],
processEscapes: true},
jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
TeX: {
extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
equationNumbers: {
autoNumber: "AMS"
}
}
});
</script>

Mixup in practise works on both the images and onehot encoding labels. However, for the purposes of this tutorial we only look at the mixup process for the images though the same process is appied to the lables.
Mixup works by sampling from $\lambda \sim Beta(\alpha, \alpha)$ and using sampled probabilty to determine the weights of the sum of the two images and their labels being mixed up.

$$ \begin{align}  
\lambda &\sim Beta(\alpha, \alpha)\\
\tilde{X} &= \lambda X_i + (1 - \lambda) X_j\\
\tilde{Y} &= \lambda Y_i + (1 - \lambda) Y_j\\
\end{align} $$

Where $\tilde{X}$ and $\tilde{Y}$ are the mixed up images and labels repectively from the images $X_i$ and $X_j$ with the assoated lables $Y_i$ and $X_j$.

Run the cell below a few times and change the play around with the *alpha* input.

In [ ]:
@_PIL_NUMPY
def mixup(image1, image2, alpha):
    lam = np.random.beta(alpha, alpha) if alpha > 0 else 1
    output = lam * image1 + (1 - lam) * image2
    return output

img1 = Image.open(IMAGE_1)
img2 = Image.open(IMAGE_2)
img = np.asarray(img)
mixup_image = mixup(img1, img2, 0.5)
mixup_image

In the mixup function you will see that it only works on image of the same size rewrite the function so it works on images of different sizes.

In [ ]:
@_PIL_NUMPY
def resized_mixup(image1, image2, alpha):
    resized_image1 = ...
    resized_image2 = ...
    lam = np.random.beta(alpha, alpha) if alpha > 0 else 1
    output = lam * resized_image1 + (1 - lam) * resized_image2
    return output

img1 = Image.open(IMAGE_1)
img3 = Image.open(IMAGE_3)
img = np.asarray(img)
resized_mixup = mixup(img1, img3, 0.5)
resized_mixup

## CutMix
CutMix is like Cutout as it remove a section from and image is like Mixup as it uses two images to make new smaple with lables. 

Instead of cutout removing several holes CutMix only removes one and replce it with the content of the secound image. It detemines the size of the cutout box using the following:

<script type="text/x-mathjax-config">
MathJax.Hub.Config({
tex2jax: {
inlineMath: [['$','$'], ['\\(','\\)']],
processEscapes: true},
jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
TeX: {
extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
equationNumbers: {
autoNumber: "AMS"
}
}
});
</script>

$$ \begin{align}  
\lambda &\sim Beta(\alpha, \alpha)\\
x_0 &\sim Uniform(0, W)\\
y_0 &\sim Uniform(0, H)\\
x_l &= W \sqrt{1 - \lambda}\\  
y_l &= H \sqrt{1 - \lambda}\\
\end{align} $$

Where $\lambda$ detrmines the width and height of the cutout based on the width W and hieght H of the image. $\lambda$ is sample from $Beta(\alpha, \alpha)$ distribution tough in CutMix experiments $\alpha = 1$. The lower left hand corner of the cutout $ (x_0, y_0) $ is uniform point detrmined by the size of the image.

In [ ]:
@_PIL_NUMPY
def cutmix(image1, image2):
    resized_image1 = ...
    resized_image2 = ...

    # sample a cutout
    h, w, _ = resized_image2.shape
    lam = np.random.uniform() # Beta(1, 1)
    cut_ratio = np.sqrt(1 - lam)
    xl, yl = w * cut_ratio, h * cut_ratio
    x0, y0 = np.random.randint(w - xl),  np.random.randint(h - yl)
    
    output = ...
    output[...] = ...
    return output

img1 = Image.open(IMAGE_1)
img2 = Image.open(IMAGE_2)
cutmix_img = cutmix(img1, img2)
cutmix_img

Make sure that your implmentation works for image of different sizes.

In [ ]:
img1 = Image.open(IMAGE_1)
img3 = Image.open(IMAGE_3)
cutmix_img = cutmix(img1, img3)
cutmix_img

<script type="text/x-mathjax-config">
MathJax.Hub.Config({
tex2jax: {
inlineMath: [['$','$'], ['\\(','\\)']],
processEscapes: true},
jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
TeX: {
extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
equationNumbers: {
autoNumber: "AMS"
}
}
});
</script>

You notice that in our *cutmix(.)* function that $x_0$ and $y_0$ are sampled slightly differently than described above. Rewrite the function and implement it so that $x_0$ and $y_0$ are sampled from the bounds of the image.

In [ ]:
@_PIL_NUMPY
def cutmix2(image1, image2):
    resized_image1 = ...
    resized_image2 = ...

    # sample a cutout
    h, w, _ = resized_image2.shape
    lam = np.random.uniform() # Beta(1, 1)
    x0, y0 = np.random.randint(w),  np.random.randint(h)
    
    cut_ratio = np.sqrt(1 - lam)
    xl, yl = w * cut_ratio, h * cut_ratio
    
    xl, yl = ...
    
    output = ...
    output[...] = ...
    return output

img1 = Image.open(IMAGE_1)
img2 = Image.open(IMAGE_2)
cutmix_img = cutmix(img1, img2)
cutmix_img